In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
PREFER_CUDA = False

In [3]:
torch.cuda.is_available()

True

In [4]:
use_cuda = PREFER_CUDA and torch.cuda.is_available()

# args.seed
batch_size = 100
nb_epochs = 5

'''kwargs = {'num_workers': 1, 'pin_memory': True} if else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   
    batch_size=, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       
    batch_size=, shuffle=True, **kwargs)
'''

vocab_size = 250
n_words = 3
n_embed = 50
n_hidden = 200
n_output = vocab_size

learning_rate = 0.1
momentum = 0.9

In [5]:
embedding = nn.Embedding(vocab_size, 8)
# a batch of 2 samples of 4 indices each
input = Variable(torch.LongTensor([[11,20,4],[30,10,20],]))
print(embedding(input))

Variable containing:
(0 ,.,.) = 
  0.6940 -0.1874 -0.1818 -0.7801 -0.1634 -1.0847  0.6530  0.3351
 -1.8435  0.8350  0.3167  1.2730 -0.8385  1.7186  0.6867 -0.9712
 -0.9578 -0.0720  0.0979 -2.5491 -0.1813  0.3828  2.3271 -0.9195

(1 ,.,.) = 
 -1.0836  1.1430 -0.6539 -0.0661  0.4463 -0.0222  1.0801 -1.8776
 -0.5286  2.1875  0.0653 -0.4930 -0.9506  0.9420  0.6597  0.1213
 -1.8435  0.8350  0.3167  1.2730 -0.8385  1.7186  0.6867 -0.9712
[torch.FloatTensor of size 2x3x8]



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convert word into vectors
        # see documentation : http://pytorch.org/docs/nn.html#torch.nn.Embedding
        self.W2E = nn.Embedding(vocab_size, n_embed, padding_idx=None, max_norm=None, norm_type=2)
        self.E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
        self.H2O = nn.Linear(n_hidden, n_output, bias=True)
        #self.conv2_drop = nn.Dropout2d()

    def forward(self, words):
        wvectors = self.W2E(words)
        # WARNING : wvectors may need a reshape operation ???
        wvectorsR = wvectors.resize(words.size(0), n_words*n_embed)
        h_activation = self.E2H(wvectorsR)
        h = F.sigmoid(h_activation)
        out_activation = self.H2O(h)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #return F.log_softmax(x)
        return F.softmax(out_activation) # WARNING : or log_softmax ???

model = Net()
if use_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [7]:
from utilities import load_data
[train_input, train_target, valid_input, valid_target, test_input, test_target, vocab] = load_data(100)
train_input.shape

utilities.py:30: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  train_input = reshape(data.trainData[0:D, 0:N * M], (D, N, M));
/home/ubuntu/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
utilities.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  train_target = reshape(data.trainData[D, 0:N * M], (1, N, M));


(3, 100, 3725)

In [8]:
TEST = True
if TEST:
    words = Variable(torch.LongTensor([[1,4,2], [13,44,3]]))
    W2E = nn.Embedding(vocab_size, n_embed, padding_idx=None, max_norm=None, norm_type=2)
    E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
    H2O = nn.Linear(n_hidden, n_output, bias=True)
    #
    wvectors = W2E(words)
    #print(wvectors.size())
    wvectorsR = wvectors.resize(words.size(0), n_words*n_embed)
    #print(wvectorsR.size())
    h_activation = E2H(wvectorsR)
    h = F.sigmoid(h_activation)
    out_activation = H2O(h)
    print(out_activation.size())

torch.Size([2, 250])


In [9]:
if TEST:
    batch_idx = 0
    data = torch.LongTensor(train_input[:,:,batch_idx].T.astype('int'))
    print(data.size())
    target = torch.LongTensor(train_target[:,:,batch_idx].T.astype('int'))
    print(target.size())
    if use_cuda:
        data = data.cuda()
        target = target.cuda()
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)

torch.Size([100, 3])
torch.Size([100, 1])


In [10]:
def train(epoch):
    model.train()
    for batch_idx in range(10):
        data = torch.LongTensor(train_input[:,:,batch_idx].T.astype('int'))
        print(data.size())
        target = torch.LongTensor(train_target[:,:,batch_idx].T.astype('int'))
        print(target.size())
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target, size_average=True) # WARNING : target one_hot_encoding
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, nb_epochs + 1):
    #train(epoch)
    #test(epoch)
    None

In [11]:
train(1)

torch.Size([100, 3])
torch.Size([100, 1])


RuntimeError: multi-target not supported at /data/users/soumith/miniconda2/conda-bld/pytorch-0.1.6_1485015011449/work/torch/lib/THNN/generic/ClassNLLCriterion.c:20